<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load" data-toc-modified-id="Load-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load</a></span></li><li><span><a href="#LGBM-training" data-toc-modified-id="LGBM-training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LGBM training</a></span><ul class="toc-item"><li><span><a href="#Prepare" data-toc-modified-id="Prepare-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare</a></span></li><li><span><a href="#Core" data-toc-modified-id="Core-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Core</a></span></li></ul></li></ul></div>

# Load

In [1]:
import taiko as tk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import classification_report

SONG = 1
WHO = 7
TRAIN_ORDER = 3

Using TensorFlow backend.


In [2]:
pfs = []
for drummer in tqdm(range(1, 8 + 1)):
    ppf = []
    for order_id in range(1, 3 + 1):
        if SONG == 4 and drummer == 4 and (order_id in [3, 4]):
            ppf.append(None)
            continue
        ppf.append(tk.get_event_primitive_df(drummer, SONG, order_id, True, True))
    pfs.append(ppf)

100%|██████████| 8/8 [01:56<00:00, 14.58s/it]


In [3]:
def soso(who_id, song_id):
    sfs = []
    for pf_order in range(1, 3 + 1):
        score = tk.get_score(who_id, song_id, pf_order)
        sfs.append(score)
    return sfs

In [4]:
sfs = []
for drummer in range(1, 8 + 1):
    sfs.append(soso(drummer, SONG))

In [5]:
sfs[0][0]

243400

In [6]:
pfs[0][2]

,hit_type,R_A_AI,R_G_AI,R_AX_AI,R_AY_AI,R_AZ_AI,R_GX_AI,R_GY_AI,R_GZ_AI,R_A_VI,...,L_AXY_CORR,L_AYZ_CORR,L_AZX_CORR,L_GXY_CORR,L_GYZ_CORR,L_GZX_CORR,L1,L2,R1,R2
0,1,2.18300,0.31090,-0.530700,-1.17100,3.29400,0.385600,0.03834,-0.04310,6.63000,...,-0.387900,-0.70760,0.63830,-0.008275,0.45070,0.76470,0,0,0,0
1,0,-0.32950,0.71270,0.115900,-0.07638,0.74880,1.967000,2.89100,-3.44500,-0.27550,...,0.342900,0.45640,0.48420,0.361700,0.68910,-0.33410,1,0,0,0
2,0,-0.25540,-0.36120,-1.528000,-0.57240,0.07543,-0.138800,1.90900,-0.33050,-0.43650,...,-0.861500,0.74190,-0.93840,-0.664000,-0.91920,0.78160,0,1,0,1
3,0,-0.12100,-0.55290,0.036020,-1.70200,-0.73090,-0.619600,1.22100,0.16370,-0.42140,...,0.981700,-0.87310,-0.86070,0.929900,-0.68270,-0.54000,0,0,1,0
4,1,0.01146,-0.51790,0.194900,-1.94400,-1.34000,0.864400,0.49000,-0.08949,-0.41700,...,0.622400,-0.18360,-0.65170,0.588800,0.14050,0.09427,0,0,0,0
5,0,-0.25540,-0.56860,0.493100,-1.07600,-1.51000,-0.436500,0.19380,-0.58310,-0.40920,...,-0.222800,0.81780,-0.62430,-0.139500,-0.34090,0.69100,1,0,0,0
6,0,-0.22920,-0.34620,-0.143500,-1.34700,-0.88650,-0.591400,-1.22000,0.88360,-0.42110,...,0.574700,0.36690,-0.46100,-0.937800,-0.93000,0.96320,0,1,0,1
7,0,-0.67980,-0.02710,0.620600,-0.11790,-0.31620,-1.586000,-1.65800,1.64100,-0.42750,...,0.918400,-0.37390,-0.24670,-0.552900,0.42340,-0.57780,0,0,1,0
8,1,0.51520,0.57370,-0.353100,0.74240,0.57890,0.178300,-1.25700,1.71600,0.52820,...,-0.709400,0.70460,-0.54220,0.620800,0.38390,-0.36500,0,0,0,1
9,0,-0.46710,-0.17050,0.709900,1.18700,0.23510,0.595300,1.94700,-1.11900,-0.39420,...,-0.538900,0.88960,-0.27490,0.135600,-0.27410,-0.88200,1,0,1,0


# LGBM training

## Prepare

In [7]:
def prepare(test_who):
    train_dfs = []
    for drummer in range(1, 8 + 1):
        if test_who == drummer:
            continue
        train_dfs.append(pfs[drummer - 1][TRAIN_ORDER - 1])
    train_df = pd.DataFrame(pd.concat(train_dfs, ignore_index=True))
    
    x = train_df.drop(['hit_type'], axis=1)
    y = train_df['hit_type']
    
    valid_df = train_df.copy()
    x_valid = valid_df.drop(['hit_type'], axis=1)
    y_valid = valid_df['hit_type']
    
    return x, y, x_valid, y_valid

In [8]:
params = dict({
    'learning_rate': 0.2,
    'application': 'multiclass',
    'num_classes': 3,
#     'min_data_in_leaf': 5,
#     'max_depth': 8,
    'num_leaves': 2 ** 4,
    'verbosity': 0,
#     'metric': 'multi_error'
})

grid_params = {
    'learning_rate': [0.1, 0.2],
    'max_depth': [8, 10],
}

In [9]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    return cm

In [10]:
def my_f1_score(y_pred, y):

    y = y.get_label()
    y_pred = y_pred.reshape(len(np.unique(y)), -1).argmax(axis=0)
    return "f1-score", metrics.f1_score(y, y_pred, average="weighted"), True

## Core

In [11]:
result_df = pd.DataFrame(
    columns=['test_who', 'test_porder', 'test_score', 'f1_score'])
id_ = 0

for test_who in range(1, 8 + 1):
    for test_porder in range(1, 3 + 1):
        x, y, x_valid, y_valid = prepare(test_who)

        train_set = lgb.Dataset(x, y)
        valid_set = lgb.Dataset(x_valid, y_valid, free_raw_data=False)

        watchlist = [valid_set]

        model = lgb.train(params, train_set=train_set, valid_sets=watchlist,
                          num_boost_round=200, verbose_eval=50, early_stopping_rounds=100, feval=my_f1_score)

        if pfs[test_who - 1][test_porder - 1] is None:
            continue
        x_test = pfs[test_who - 1][test_porder -
                                   1].drop(['hit_type'], axis=1)
        y_true = pfs[test_who - 1][test_porder - 1]['hit_type']

        y_pred = model.predict(x_test, num_iteration=model.best_iteration)

        predictions = []
        for xx in y_pred:
            predictions.append(np.argmax(xx))
        y_test = pd.Series(data=predictions)

        f1_score = '%.4f' % metrics.f1_score(
            y_test, y_true, average='weighted')
        
        print(classification_report(y_true, y_test))
        
        test_score = sfs[test_who - 1][test_porder - 1]

        result_df.loc[id_] = [test_who, test_porder, test_score, f1_score]

        id_ += 1

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0's f1-score: 1
Early stopping, best iteration is:
[28]	valid_0's f1-score: 1
             precision    recall  f1-score   support

          0       0.95      0.95      0.95       238
          1       0.88      0.90      0.89        93
          2       1.00      0.90      0.95        30

avg / total       0.94      0.94      0.94       361

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0's f1-score: 1
Early stopping, best iteration is:
[28]	valid_0's f1-score: 1
             precision    recall  f1-score   support

          0       0.97      0.95      0.96       238
          1       0.89      0.97      0.93        93
          2       1.00      0.83      0.91        30

avg / total       0.95      0.95      0.95       361

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0'

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0's f1-score: 1
Early stopping, best iteration is:
[33]	valid_0's f1-score: 1
             precision    recall  f1-score   support

          0       0.98      0.98      0.98       245
          1       0.97      0.97      0.97        86
          2       0.91      0.97      0.94        30

avg / total       0.97      0.97      0.97       361

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0's f1-score: 1
Early stopping, best iteration is:
[33]	valid_0's f1-score: 1
             precision    recall  f1-score   support

          0       0.98      0.96      0.97       245
          1       0.95      0.94      0.95        86
          2       0.85      0.97      0.91        30

avg / total       0.96      0.96      0.96       361

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's f1-score: 1
[100]	valid_0'

In [12]:
result_df.to_csv('CSV/song1_result-sc-yy.csv', index=False)